In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!dir

In [33]:
df_train = pd.read_csv('train_hire_stats.csv')
df_train.shape

(219600, 4)

In [ ]:
tt=pd.to_datetime(df_train["Date"]).head()

In [34]:
df_train["tdate"] = pd.to_datetime(df_train["Date"]).dt.date

In [83]:
df_train.shape

(219600, 5)

In [87]:
d2=df_train.drop(['Date'], axis=1)
d2.shape

(219600, 4)

In [88]:
rowlen(d2)

219600

In [89]:
d2["daytype"] = np.ones((len(d2),), dtype=int)

In [91]:
d2.head()

,Zone_ID,Hour_slot,Hire_count,tdate,daytype
0,1,0,0,2016-02-01,1
1,1,1,0,2016-02-01,1
2,1,2,0,2016-02-01,1
3,1,3,0,2016-02-01,1
4,1,4,0,2016-02-01,1


In [86]:
df_train.head()

,Zone_ID,Date,Hour_slot,Hire_count,tdate
0,1,2016-02-01,0,0,2016-02-01
1,1,2016-02-01,1,0,2016-02-01
2,1,2016-02-01,2,0,2016-02-01
3,1,2016-02-01,3,0,2016-02-01
4,1,2016-02-01,4,0,2016-02-01


In [ ]:
df2.iloc[88]  

In [38]:
zone18 = df_train[df_train['Zone_ID']==18]
zone18.shape

(8784, 5)

In [74]:
zone18_date = zone18[['tdate']]


(8784, 1)

In [77]:
print(zone18_date.iloc[0]['tdate'].weekday())

0


In [ ]:
#dd.append(pd.Timestamp.weekday(row[0].date.weekday()))
#.weekday(), sep='x'
#print(datetime.strptime(row[0], dateFormatter).weekday())
#zone18_date.iloc[0]
#from datetime import datetime
#dateFormatter = "%Y-%m-%d"

In [82]:
dd = []
for index, row in zone18_date.iterrows():
    dd.append(row['tdate'].weekday())
dd[0:3]

[0, 0, 0]

In [54]:
zone18_holidays = pd.to_datetime(zone18['Date'])
pd.Timestamp.weekday(zone18_holidays.iloc[88])

3

In [40]:
zone18['tdate'].head()

149328    2016-02-01
149329    2016-02-01
149330    2016-02-01
149331    2016-02-01
149332    2016-02-01
Name: tdate, dtype: object

In [39]:
zone18_holidays = zone18[zone18['tdate'].weekday()>=5]
zone18_holidays.shape

AttributeError: 'Series' object has no attribute 'weekday'

In [ ]:
zone18 = df_train[df_train['tdate']==18]

In [ ]:
import datetime
day_Monday = datetime.date(2016, 2, 1)
print(day_Monday.weekday())
print(day_Monday.strftime('%A'))

In [ ]:
day_Tuesday = day_Monday + datetime.timedelta(days=1)
print(day_Tuesday.weekday())
print(day_Tuesday.strftime('%A'))

In [ ]:
df2.iloc[88].weekday() 

In [ ]:
#0 == Monday
#1 == Tuesday
#2 == Wednesday
#3 == Thursday
#4 == Friday
#5 == Saturday
#6 == Sunday

df2.iloc[88].strftime('%A')

In [ ]:
zone18 = df_train[df_train['Zone_ID']==18]
zone18.shape

In [ ]:

plt.plot(zone18['Hire_count'][168:330])

In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
import time #helper libraries

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# fix random seed for reproducibility
np.random.seed(5)

In [ ]:
all_y = zone18['Hire_count'].values
dataset=all_y.reshape(-1, 1)
print(dataset.shape)

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets, 50% test data, 50% training data
train_size = int(len(dataset) * 0.5)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# reshape into X=t and Y=t+1, timestep 240
look_back = 168
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network, optimizer=adam, 25 neurons, dropout 0.1
model = Sequential()
model.add(LSTM(25, input_shape=(1, look_back)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=240, verbose=1)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
train_holidays=['2016/2/8', '2016/2/9', '2016/2/10', '2016/2/11', '2016/2/12',
'2016/2/29', '2016/4/4', '2016/4/5', '2016/6/9', '2016/6/10', 
'2016/9/15', '2016/9/16', '2016/10/10']

train_workdays = ['2016/6/4', '2016/9/10']
test_holidays = ['2017/2/1','2017/2/27', '2017/2/28']
test_workdays = ['2017/2/18']
